In [2]:
import pandas as pd
import ast
import pyarrow.parquet as pq
import ast
import time
import gzip
%load_ext memory_profiler

In [ ]:
#items = pd.read_csv(r'../data_sets/items_user.csv',index_col=False)

In [3]:
item = pd.read_csv(r'../data_set_limpio/item_desplegado.csv.gz')

In [4]:
df = pd.read_csv(r'../data_set_limpio/games_preparado.csv.gz')

In [ ]:
#items = items[items['items'] != '[]'] # ---> realizo una limpieza de los usuarios que no tienen items, por que no van a ser relevantes.

In [10]:
item.head()

,item_id,item_name,playtime_forever,user
0,10,Counter-Strike,6,76561197970982479
1,30,Day of Defeat,7,76561197970982479
2,300,Day of Defeat: Source,4733,76561197970982479
3,240,Counter-Strike: Source,1853,76561197970982479
4,3830,Psychonauts,333,76561197970982479


In [ ]:
user = '76561197970982479'

usuario = item.loc[item['user_id'] == user]['items']

if not usuario.empty:
    usuario = usuario.iloc[0]
data = ast.literal_eval(usuario)
result = pd.DataFrame(data)
result.drop(columns='playtime_2weeks',inplace=True)

result = result[result['playtime_forever'] != 0]
result['user'] = user
result

In [ ]:
def process_data_first_100(item):
    results = []  # Lista para almacenar los resultados desanidados
    limited_item = item.head(1000)  # Tomar solo las primeras 100 filas

    start_time = time.time()  # Registrar el tiempo de inicio

    for index, row in limited_item.iterrows():
        usuario = row['items']

        if not pd.isnull(usuario):  # Verifica si el valor no es nulo
            usuario = ast.literal_eval(usuario)
            result = pd.DataFrame(usuario)
            result.drop(columns='playtime_2weeks', inplace=True)
            result = result[result['playtime_forever'] != 0]
            result['user'] = row['user_id']
            results.append(result)

    end_time = time.time()  # Registrar el tiempo al finalizar

    execution_time = end_time - start_time  # Calcular el tiempo de ejecución
    print(f"Tiempo de ejecución: {execution_time} segundos")

    concatenated_results = pd.concat(results, ignore_index=True) if results else pd.DataFrame()  # Concatenar resultados

    return concatenated_results


In [ ]:
def process_data_full(item):
    results = []  # Lista para almacenar los resultados desanidados

    start_time = time.time()  # Registrar el tiempo de inicio

    for index, row in item.iterrows():
        usuario = row['items']

        if not pd.isnull(usuario):  # Verifica si el valor no es nulo
            usuario = ast.literal_eval(usuario)
            result = pd.DataFrame(usuario)
            result.drop(columns='playtime_2weeks', inplace=True)
            result = result[result['playtime_forever'] != 0]
            result['user'] = row['user_id']
            result
        
            results.append(result)

    end_time = time.time()  # Registrar el tiempo al finalizar

    execution_time = end_time - start_time  # Calcular el tiempo de ejecución
    print(f"Tiempo de ejecución: {execution_time} segundos")

    concatenated_results = pd.concat(results, ignore_index=True) if results else pd.DataFrame()  # Concatenar resultados

    return concatenated_results


In [ ]:
resultados_concatenados = process_data_full(item)

In [30]:
df = df[['id','price']]

In [39]:
item_precio = pd.merge(item, df, how='left', left_on='item_id', right_on='id')

item_precio.dropna(inplace=True)

item_precio

,item_id,item_name,playtime_forever,user,id,price
0,10,Counter-Strike,6,76561197970982479,10.0,9.99
1,30,Day of Defeat,7,76561197970982479,30.0,4.99
2,300,Day of Defeat: Source,4733,76561197970982479,300.0,9.99
3,240,Counter-Strike: Source,1853,76561197970982479,240.0,19.99
4,3830,Psychonauts,333,76561197970982479,3830.0,9.99
...,...,...,...,...,...,...
3285240,417860,Emily is Away,8,76561198323066619,417860.0,0.00
3285241,304930,Unturned,677,76561198329548331,304930.0,0.00
3285242,227940,Heroes & Generals,43,76561198329548331,227940.0,0.00
3285243,388490,One Way To Die: Steam Edition,3,76561198329548331,388490.0,0.00


In [40]:
item_precio.to_csv('../data_set_limpio/item_desplegado.csv',encoding='UTF-8',index=False)

In [41]:
with open('../data_set_limpio/item_desplegado.csv', 'rb') as f_in, gzip.open('../data_set_limpio/item_desplegado.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)

In [ ]:
solo_necesario = item[['user_id','items_count']]

In [ ]:
solo_necesario.to_csv('../data_set_limpio/item_cantidad_usuarios.csv',encoding='UTF-8',index=False)

In [ ]:
with open('../data_set_limpio/item_cantidad_usuarios.csv', 'rb') as f_in, gzip.open('../data_set_limpio/item_cantidad_usuarios.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)